# List of active United States Military Aircraft

This is a project to visualize a list of **active** military aircraft that are used by the United States military. This will not include aircraft that is no longer in-service.

### Contents:
- Business Request & User Stories
- Data Cleansing & Transformation (Python)
- Data Model
- Active US Military Aircraft Dashboard

## Business Request & User Stories 

The business request for this data analyst project was an executive military aircraft report for enthusiasts. Based on the request that was made from the business (aka me), the following user stories were defined to fulfill delivery and ensure that acceptance criteria was maintained throughout the project.


| #	As a (role) | I want (request / demand) | So that I (user value) | Acceptance Criteria
| :- | :- | :- | :- 
| 1. Military dude | To get a dashboard overview of active US military aircraft | Can follow better what aircraft is in use | A Power BI dashboard which updates data upon refresh
| 2. Veteran | A detailed overview of Aircraft per Branch | Can follow what aircraft is in use by each branch | A Power BI dashboard which allows me to filter data for each branch
| 3. Sales Representative | A detailed overview of Aircraft per Manufacturer | Can follow up which Manufacturer and aircraft is most in use | A Power BI dashboard which allows me to filter data for each Manufacturer
| 4. Time Traveler | A dashboard overview of Aircraft introduction dates | Follow aircraft in service over date introduced | A Power Bi dashboard with graphs to filter by dates ascending

## Data Cleansing & Transformation (Python)

To create the necessary data model for doing analysis and fulfilling the business needs defined in the user stories, we will be using pandas, requests, and beautifulsoup to webscrape a wikipedia page.

Source: https://en.wikipedia.org/wiki/List_of_active_United_States_military_aircraft

In [1]:
import pandas as pd
import requests
import numpy as np
import re
from bs4 import BeautifulSoup

In [2]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_active_United_States_military_aircraft').text
soup = BeautifulSoup(page, 'html.parser')
table = soup.find_all('table', class_="wikitable sortable")

df = pd.read_html(str(table))
df = pd.concat(df)
print(df.shape)

(162, 14)


### Reading data in pandas

In [3]:
# Read headers
df.columns

Index(['Aircraft', 'Manufacturer', 'Origin', 'Propulsion', 'Role', 'Control',
       'Introduced', 'In service[1][2][3]', 'Total', 'Notes', 'Type',
       'In service[15]', 'In service[22]', 'In service[23][24]'],
      dtype='object')

In [4]:
# !pip install -U klib

import klib

# Clean columns using klib library
df = klib.clean_column_names(df) # cleans and standardizes column names
df = klib.convert_datatypes(df) # converts existing to more efficient dtypes

df.columns
df.dtypes

aircraft                 string
manufacturer             string
origin                   string
propulsion             category
role                     string
control                category
introduced               object
in_service[1][2][3]      string
total                    object
notes                    string
type                     string
in_service[15]           string
in_service[22]          float32
in_service[23][24]       string
dtype: object

### Make changes to the data

In [5]:
# Assign values from "type" column if "aircraft" is blank
df.aircraft = df.aircraft.replace('', pd.NA).fillna(df.type)
df.tail()

,aircraft,manufacturer,origin,propulsion,role,control,introduced,in_service[1][2][3],total,notes,type,in_service[15],in_service[22],in_service[23][24]
25,MH-60 Seahawk,Sikorsky,USA,Helicopter,Anti-submarine warfare helicopterMulti-mission,Manned,1984,<NA>,NaN,<NA>,MH-60 Seahawk,508,NaN,<NA>
26,TH-57B/C Sea Ranger,Bell,USA,Helicopter,Trainer,Manned,1984,<NA>,NaN,To be replaced by the AgustaWestland TH-57A Th...,TH-57B/C Sea Ranger,114,NaN,<NA>
27,ScanEagle,Boeing,USA,Propeller,<NA>,Unmanned,2005,<NA>,NaN,<NA>,ScanEagle,<NA>,NaN,<NA>
28,RQ-21A Blackjack,Boeing Insitu,USA,Propeller,<NA>,Unmanned,2014,<NA>,NaN,<NA>,RQ-21A Blackjack,<NA>,NaN,<NA>
29,MQ-8B Fire Scout,Northrop Grumman,USA,Helicopter,Patrol,Unmanned,2009,<NA>,NaN,Helicopter. 96 planned.[33],MQ-8B Fire Scout,27[27],NaN,<NA>


In [6]:
# Review dtypes before working with re to clean the dataframe
df.dtypes

aircraft                 string
manufacturer             string
origin                   string
propulsion             category
role                     string
control                category
introduced               object
in_service[1][2][3]      string
total                    object
notes                    string
type                     string
in_service[15]           string
in_service[22]          float32
in_service[23][24]       string
dtype: object

In [7]:
# We see that one column is a "Float32", so we need to convert 'in_service[22]' column into a string
df['in_service[22]'] = df['in_service[22]'].astype('string')

df.dtypes

aircraft                 string
manufacturer             string
origin                   string
propulsion             category
role                     string
control                category
introduced               object
in_service[1][2][3]      string
total                    object
notes                    string
type                     string
in_service[15]           string
in_service[22]           string
in_service[23][24]       string
dtype: object

In [8]:
# Replace all NaN values with blank
df = df.replace(np.nan, '', regex=True)

In [9]:
# Combine in_service[x] columns into one
cols = ['in_service[1][2][3]', 'in_service[15]', 'in_service[22]', 'in_service[23][24]']
df['in_service'] = df[cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)

# Remove square brackets that were intended for footnotes in the wiki; This is done by using a delimiter
df['in_service'] = df['in_service'].str.split('[').str[0]

# Drop columns
df = df.drop(columns=cols)
df.drop('type', axis=1, inplace=True) # This was merged with aircraft column

df.head()

,aircraft,manufacturer,origin,propulsion,role,control,introduced,total,notes,in_service
0,A-10C Thunderbolt II,Fairchild Republic,USA,Jet,CAS / Attack,Manned,1977,,,281
1,AC-130J Ghostrider,Lockheed,USA,Propeller,CAS / Attack,Manned,2017,,Replacement for the AC-130U.,6
2,AC-130W Stinger II,Lockheed,USA,Propeller,CAS / Attack,Manned,1966,,Currently being replaced by the AC-130J.,20
3,B-1B Lancer,Rockwell International,USA,Jet,Bomber,Manned,1986,,Employs variable-sweep wing design. To be repl...,45
4,B-2A Spirit,Northrop Grumman,USA,Jet,Bomber,Manned,1997,,Stealth capable aircraft. To be replaced by th...,19


In [10]:
# Create a new column called 'branch'
df['branch'] = ''

# Assign branch based on index location; 'branch' column is [10]
# !! Compare with wiki page in case something is added/removed (since this uses index loc) !!
df.iloc[0:66,[10]] = 'Air Force'
df.iloc[66:94,[10]] = 'Army'
df.iloc[94:104,[10]] = 'Coast Guard'
df.iloc[104:132,[10]] = 'Marines'
df.iloc[132:162,[10]] = 'Navy'

df


,aircraft,manufacturer,origin,propulsion,role,control,introduced,total,notes,in_service,branch
0,A-10C Thunderbolt II,Fairchild Republic,USA,Jet,CAS / Attack,Manned,1977,,,281,Air Force
1,AC-130J Ghostrider,Lockheed,USA,Propeller,CAS / Attack,Manned,2017,,Replacement for the AC-130U.,6,Air Force
2,AC-130W Stinger II,Lockheed,USA,Propeller,CAS / Attack,Manned,1966,,Currently being replaced by the AC-130J.,20,Air Force
3,B-1B Lancer,Rockwell International,USA,Jet,Bomber,Manned,1986,,Employs variable-sweep wing design. To be repl...,45,Air Force
4,B-2A Spirit,Northrop Grumman,USA,Jet,Bomber,Manned,1997,,Stealth capable aircraft. To be replaced by th...,19,Air Force
...,...,...,...,...,...,...,...,...,...,...,...
25,MH-60 Seahawk,Sikorsky,USA,Helicopter,Anti-submarine warfare helicopterMulti-mission,Manned,1984,,,508,Navy
26,TH-57B/C Sea Ranger,Bell,USA,Helicopter,Trainer,Manned,1984,,To be replaced by the AgustaWestland TH-57A Th...,114,Navy
27,ScanEagle,Boeing,USA,Propeller,,Unmanned,2005,,,,Navy
28,RQ-21A Blackjack,Boeing Insitu,USA,Propeller,,Unmanned,2014,,,,Navy


In [11]:
# Filter out any letters in the 'introduced' column (ex. 1980s)
df.introduced = df.introduced.astype('string')
df.introduced = df.introduced.str.extract('([0-9]+[0-9]+[0-9]+[0-9])')

### Filtering data

In [12]:
# Example: Find Aircraft made by Lockheed
lockheed = df.loc[df['manufacturer'].str.contains("Lockheed")]
lockheed

,aircraft,manufacturer,origin,propulsion,role,control,introduced,total,notes,in_service,branch
1,AC-130J Ghostrider,Lockheed,USA,Propeller,CAS / Attack,Manned,2017,,Replacement for the AC-130U.,6,Air Force
2,AC-130W Stinger II,Lockheed,USA,Propeller,CAS / Attack,Manned,1966,,Currently being replaced by the AC-130J.,20,Air Force
6,C-5M Super Galaxy,Lockheed,USA,Jet,Strategic airlifter,Manned,1970,,,52,Air Force
13,C-130H Hercules,Lockheed Martin,USA,Propeller,Tactical airlifter,Manned,1956,,,192,Air Force
14,C-130J Super Hercules,Lockheed Martin,USA,Propeller,Tactical airlifter,Manned,1996,,,132,Air Force
24,EC-130H Compass Call,Lockheed Martin,USA,Propeller,Radar jamming / PSYOP,Manned,1982,,,12,Air Force
25,EC-130J Commando Solo III,Lockheed Martin,USA,Propeller,Radar jamming / PSYOP,Manned,1978,,,7,Air Force
30,F-22A Raptor,Lockheed Martin,USA,Jet,Air superiority,Manned,2005,,Stealth capable aircraft. To be replaced by NGAD,186,Air Force
31,F-35A Lightning II,Lockheed Martin,USA,Jet,Multirole,Manned,2016,,Stealth capable aircraft. F-16 replacement,283,Air Force
32,HC-130J Combat King II,Lockheed Martin,USA,Propeller,Search and rescue,Manned,1959,,,32,Air Force


### Write the dataframe to parquet format.

After preprocessing data using Python, we can convert the dataframe to a format that can be loaded into a visualization tool (e.g., Microsoft PowerBI).


In [13]:
#!pip install fastparquet

df.to_excel('aircraft.xlsx')
